In [1]:
import importlib
from dbm import error
import ast
import GeeksCalculator as gk
import pandas as pd
import numpy as np
import warnings
import plotly.graph_objects as go
import pandas as pd
import plotly.graph_objs as go
from pandas import date_range, notna
from pandas.errors import SettingWithCopyWarning
from plotly.subplots import make_subplots
import functions_straddle as fn
import process as dp
import plotly.io as pio
import psycopg2
dt = 'date_timestamp'
conn = psycopg2.connect(dbname="qdap_test",user="amt",password="your_password",host="192.168.2.23",port="5432"  )
cursor = conn.cursor()
# conn = psycopg2.connect(dbname="qdap_test",user="amt",password="your_password",host="192.168.2.23",port="5432"  )
# cursor = conn.cursor()
importlib.reload(dp); importlib.reload(fn)
warnings.simplefilter(action='ignore', category=FutureWarning);warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

In [ ]:
symbol = 'BANKNIFTY'
futures_expiries = {}
list_symbol = ['BANKNIFTY', 'NIFTY', 'MIDCPNIFTY']

# Load future expiry data into futures_expiries
for sym in list_symbol:
    temp = pd.read_csv(f'C:\\Users\\Tanmay\\Desktop\\Files\\db\\futures_expiries_{sym}.csv')
    futures_expiries[sym] = {}
    for i, row in temp.iterrows():
        if isinstance(row['expiry'], str):
            print(sym,row['expiry'][0:7])
            futures_expiries[sym][row['expiry'][0:7]] = row['expiry']

# Load option expiry data into options_expiries
options_expiries = {}
for sym in list_symbol:
    temp = pd.read_csv(f'C:\\Users\\Tanmay\\Desktop\\Files\\db\\expiry_IW1_{sym}.csv')
    for i, row in temp.iterrows():
        if isinstance(row['expiry'], str):
            options_expiries[row['expiry']] = (sym, row['expiry'])

In [ ]:

ranges = pd.date_range(start='2020-05-19', end='2020-12-14')
all_trades = []
all_execs = []

importlib.reload(dp); importlib.reload(fn)

for d_t in ranges:
    try:
        futures = dp.FetchFuturesByDate(cursor,symbol,d_t)
        if not futures.empty:
            options = dp.FetchOptionsByDate(cursor,symbol,d_t)
            print(str(d_t.date()))
            options['delta'] = 0.0;options['gamma']  = 0.0;options['vega'] = 0.0;options['theta'] = 0.0;options['iv'] = 0.0;options['rho'] = 0.0
            for i,row in options.iterrows():
                call_strikes = options[(options['date_timestamp']== row[dt])  & (options['opt_type']=='CE')].sort_values(by='strike')
                put_strikes = options[(options['date_timestamp'] == row[dt]) & (options['opt_type']=='CE')].sort_values(by='strike')
                SpotPrice = futures[futures[dt]==row[dt]].iloc[0]['close']
                syn_fut = fn.syn_future(call_strikes, put_strikes,SpotPrice)
                greeks = gk.calc(syn_fut,row['strike'],6,row['close'],row['expiry'],0,row['date_timestamp'],row['opt_type'])
    
                options.at[i, 'delta'] = greeks['delta']
                if greeks['delta'] is not None:
                    options.at[i, 'delta'] = round(greeks['delta'],3)
                options.at[i, 'gamma'] = greeks['gamma']
                options.at[i, 'vega'] = greeks['vega']
                options.at[i, 'theta'] = greeks['theta']
                options.at[i, 'iv'] = greeks['implied_volatility']  
                options.at[i, 'rho'] = greeks['rho']
            options.to_csv(f'{d_t.date()}.csv')
            gen_call,gen_put = options[options['opt_type']=='CE'] , options[options['opt_type']=='PE']
            call,put = dp.fetch_call_put_(options)
            call.ffill(inplace=True) ; put.ffill(inplace=True)
            execs,trades, running_pnl, net_pnl, capital, rollover, positions, difference, call_movement, put_movement, delta_put, delta_call = fn.generate_trades_syn(d_t.date(),futures, call, put, gen_call, gen_put, io=0,delta=0.2,delta_range=0.1,delta_upper_bound=0.55, buying='close', window=5, selling='close', initial_capital=100000)
            all_trades.extend(trades)
            all_execs.extend(execs)
            if len(trades) == 0:
                print(f"No trades took place on {d_t}")
            # all_execs.append({
            # 'id':None,'time': None,'opt_type':None,'log':None,'delta':None,'strike': None, 'price': None,'pnl': None, 'trade_type': None,'transaction_cost': None, 'slippage': None,
            # })
            # streams = [running_pnl,call_movement,put_movement,delta_call,delta_put,difference]
            # labels = ['running pnl','call_strike','put_strike','delta_call','delta_put','diff']
            # dp.plotter(streams = streams,labels = labels,filename=f'Graph_{d_t.date()}.html')
    except:
        print(f"Error in date {d_t}")
        